# Module 2 Assessment

Welcome to your Mod 2 Assessment. You will be tested for your understanding of concepts and ability to solve problems that have been covered in class and in the curriculum.

Use any libraries you want to solve the problems in the assessment.

You will have up to two hours to complete this assessment.

The sections of the assessment are:

- Accessing Data Through APIs
- Object Oriented Programming
- SQL and Relational Databases
- HTML, CSS and Web Scraping
- Other Database Structures (MongoDB)

In this assessment you will be exploring two datasets: Pokemon and Quotes.

In [2]:
# import the necessary libraries

import requests
import json
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup
# import pymongo

## Part 1: Accessing Data Through APIs

In this section we'll be using PokeAPI to get data on Pokemon. Let's first define functions to get information from the API. Provided below is a URL that will get you started with the first 151 Pokemon! Run the cell below to see what we get.

In [56]:
url = 'https://pokeapi.co/api/v2/pokemon/?limit=151'
results = requests.get(url).json()['results']
results

[{'name': 'bulbasaur', 'url': 'https://pokeapi.co/api/v2/pokemon/1/'},
 {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'},
 {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon/3/'},
 {'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon/4/'},
 {'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'},
 {'name': 'charizard', 'url': 'https://pokeapi.co/api/v2/pokemon/6/'},
 {'name': 'squirtle', 'url': 'https://pokeapi.co/api/v2/pokemon/7/'},
 {'name': 'wartortle', 'url': 'https://pokeapi.co/api/v2/pokemon/8/'},
 {'name': 'blastoise', 'url': 'https://pokeapi.co/api/v2/pokemon/9/'},
 {'name': 'caterpie', 'url': 'https://pokeapi.co/api/v2/pokemon/10/'},
 {'name': 'metapod', 'url': 'https://pokeapi.co/api/v2/pokemon/11/'},
 {'name': 'butterfree', 'url': 'https://pokeapi.co/api/v2/pokemon/12/'},
 {'name': 'weedle', 'url': 'https://pokeapi.co/api/v2/pokemon/13/'},
 {'name': 'kakuna', 'url': 'https://pokeapi.co/api/v2/pokemon/14/'},
 {'name': '

[Read the documentation here](https://pokeapi.co/) for information on navigating this API and use the API to obtain data to answer the following questions.

### Accessing Data

1. For any **one** Pokemon, retrieve the following information in a dictionary format with the following keys:
    - ID
    - Name
    - Base experience
    - Weight
    - Height
    - Types
    - Abilities

For `Types` and `Abilities`, you might want to write helper functions to have each attribute just be a list of types and a list of abilities. Your output should look like this:

```
{'id': 1, 
'name': 'bulbasaur', 
'base_experience': 64, 
'weight': 69, 
'height': 7, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}

```
    


In [23]:
# check what info I have for one pokemon
url_poke= 'https://pokeapi.co/api/v2/pokemon/1/'
data_one_poke = requests.get(url_poke).json()
data_one_poke

{'abilities': [{'ability': {'name': 'chlorophyll',
    'url': 'https://pokeapi.co/api/v2/ability/34/'},
   'is_hidden': True,
   'slot': 3},
  {'ability': {'name': 'overgrow',
    'url': 'https://pokeapi.co/api/v2/ability/65/'},
   'is_hidden': False,
   'slot': 1}],
 'base_experience': 64,
 'forms': [{'name': 'bulbasaur',
   'url': 'https://pokeapi.co/api/v2/pokemon-form/1/'}],
 'game_indices': [{'game_index': 1,
   'version': {'name': 'white-2',
    'url': 'https://pokeapi.co/api/v2/version/22/'}},
  {'game_index': 1,
   'version': {'name': 'black-2',
    'url': 'https://pokeapi.co/api/v2/version/21/'}},
  {'game_index': 1,
   'version': {'name': 'white',
    'url': 'https://pokeapi.co/api/v2/version/18/'}},
  {'game_index': 1,
   'version': {'name': 'black',
    'url': 'https://pokeapi.co/api/v2/version/17/'}},
  {'game_index': 1,
   'version': {'name': 'soulsilver',
    'url': 'https://pokeapi.co/api/v2/version/16/'}},
  {'game_index': 1,
   'version': {'name': 'heartgold',
    'ur

In [24]:
# print the keys to find our to access the requested info
data_one_poke.keys()

dict_keys(['abilities', 'base_experience', 'forms', 'game_indices', 'height', 'held_items', 'id', 'is_default', 'location_area_encounters', 'moves', 'name', 'order', 'species', 'sprites', 'stats', 'types', 'weight'])

In [42]:
results['abilities']

[{'ability': {'name': 'chlorophyll',
   'url': 'https://pokeapi.co/api/v2/ability/34/'},
  'is_hidden': True,
  'slot': 3},
 {'ability': {'name': 'overgrow',
   'url': 'https://pokeapi.co/api/v2/ability/65/'},
  'is_hidden': False,
  'slot': 1}]

In [43]:
 # define helper for types
def get_list_types (result_url):
    types_list = []
    for typ in result_url['types']:
        types_list.append(typ['type']['name'])
    return types_list 

#  define helper for abilities
def get_list_abilities (result_url):
    abilities_list = []
    for ability in result_url['abilities']:
        abilities_list.append(ability['ability']['name'])
    return abilities_list       


In [46]:
get_list_types(data_one_poke)
get_list_abilities(data_one_poke)

['chlorophyll', 'overgrow']

In [53]:
# defining a funtion to return a dictionary with info of one pokemon
def get_pokedata(url):
    
    """
    url is a string of the URL that will get you the relevant info from the API
    
    this function should return the dictionary for ONE pokemon
    
    """
    #access the url
    one_poke = requests.get(url).json()
    
    #use the key of the api response and helpers to associate a value to each key
    dict_poke = {'id': one_poke['id'],'name': one_poke['forms'][0]['name'], 
                'base_experience': one_poke['base_experience'], 
                'weight': one_poke['weight'], 
                'height': one_poke['height'], 
                'types': get_list_types(one_poke), 
                'abilities': get_list_abilities(one_poke)} 
    return dict_poke  


In [54]:
# test funtion to create dictionary for one pokemon
url1 = 'https://pokeapi.co/api/v2/pokemon/1/'
dict_one_poke = get_pokedata(url1)
print(dict_one_poke)

{'id': 1, 'name': 'bulbasaur', 'base_experience': 64, 'weight': 69, 'height': 7, 'types': ['poison', 'grass'], 'abilities': ['chlorophyll', 'overgrow']}


### Pagination

2. Get the same information for the first **151** Pokemon as a list of dictionaries ordered by Pokemon ID. Print the first and last elements of the list. (Hint: Use pagination) Your output should save the list to a variable and look like this:

```
[{'id': 1, 
'name': 'bulbasaur', 
'base_experience': 64, 
'weight': 69, 
'height': 7, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}, 
{'id': 2, 
'name': 'ivysaur', 
'base_experience': 142, 
'weight': 130, 
'height': 10, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}, ... ]

```



In [61]:
""" 

Assign to "pokedata" the list of 151 dictionaries.
You may use your function from the previous question.

"""
#codes to create a list with all 151 pokemon I got the urls for
pokedata = []
#for loop to access each individual dictionary in results
for x in range(151):
#get the url value before using the function for all the info
    url_tobe_used = results[x]['url']
    infopoke = get_pokedata(url_tobe_used)
    pokedata.append(infopoke)

In [62]:
# printing first and last elements

print(pokedata[0], pokedata[-1])

{'id': 1, 'name': 'bulbasaur', 'base_experience': 64, 'weight': 69, 'height': 7, 'types': ['poison', 'grass'], 'abilities': ['chlorophyll', 'overgrow']} {'id': 151, 'name': 'mew', 'base_experience': 270, 'weight': 40, 'height': 4, 'types': ['psychic'], 'abilities': ['synchronize']}


## Part 2: Object Oriented Programming

We're going to use the data gathered in the previous section on APIs for this section on Object Oriented Programming to instantiate Pokemon objects and write instance methods.

### Creating a Class

1. Create a class called `Pokemon` with an `__init__` method to instantiate the following attributes:
    - ID
    - Name
    - Base experience
    - Weight
    - Height
    - Types
    - Abilities
    



In [ ]:
# if you were unable to get the data from the API in the right format,
# uncomment the code below to access a JSON file with the list of dictionaries

# with open('data/pokemon.json') as f:  
#     pokelist = json.load(f)

In [97]:
"""
Create your class below with the correct syntax, including an __init__ method.

"""
class Pokemon:
    def __init__(self, ID, name, base_experience, weight, height, types, abilities):
        self.ID = ID
        self.name = name
        self.base_experience = base_experience
        self.weight = weight
        self.height = height
        self.types = types
        self.abilities = abilities
    def bmi(self):
        return round ((self.weight/10) / ((self.height/10)*(self.height/10)),2)
        


    
### Instantiating Objects

2. Using the data you obtained from the API, instantiate the first, fourth and seventh Pokemon. Assign them to the variables `bulbasaur`, `charmander` and `squirtle`.

In [98]:
# Your code here
# pokemon_to_update = ['bulbasaur', 'charmander', 'squirtle']
# poke_index = [0,3,6]
# pokemon_object_list = []
# for pokemon in pokemon_to_update:
#     for i in poke_index:
#         pokemon = Pokemon(pokedata[i]['id'],pokedata[i]['name'],pokedata[i]['base_experience'],pokedata[i]['weight'],
#                           pokedata[i]['height'],pokedata[i]['types'],pokedata[i]['abilities']) 
#     pokemon_object_list.append(pokemon)

bulbasaur = Pokemon(pokedata[0]['id'],pokedata[0]['name'],pokedata[0]['base_experience'],pokedata[0]['weight'],
                        pokedata[0]['height'],pokedata[0]['types'],pokedata[0]['abilities'])

charmander = Pokemon(pokedata[3]['id'],pokedata[3]['name'],pokedata[3]['base_experience'],pokedata[3]['weight'],
                        pokedata[3]['height'],pokedata[3]['types'],pokedata[3]['abilities'])

squirtle = Pokemon(pokedata[6]['id'],pokedata[6]['name'],pokedata[6]['base_experience'],pokedata[6]['weight'],
                        pokedata[6]['height'],pokedata[6]['types'],pokedata[6]['abilities'])                    



In [85]:
bulbasaur.types

['poison', 'grass']

In [94]:
# run this cell to test and check your code
# you may need to edit the attribute variable names if you named them differently!

def print_pokeinfo(pokemon_object):
    o = pokemon_object
    print('ID: ' + str(o.ID) + '\n' +
          'Name: ' + o.name.title() + '\n' +
          'Base experience: ' + str(o.base_experience) + '\n' +
          'Weight: ' + str(o.weight) + '\n' +
          'Height: ' + str(o.height) + '\n' +
          'Types: ' + str(o.types) + '\n' +
          'Abilities: ' + str(o.abilities) + '\n'
         )
    
print_pokeinfo(bulbasaur)
print_pokeinfo(charmander)
print_pokeinfo(squirtle)

ID: 1
Name: Bulbasaur
Base experience: 64
Weight: 69
Height: 7
Types: ['poison', 'grass']
Abilities: ['chlorophyll', 'overgrow']

ID: 4
Name: Charmander
Base experience: 62
Weight: 85
Height: 6
Types: ['fire']
Abilities: ['solar-power', 'blaze']

ID: 7
Name: Squirtle
Base experience: 63
Weight: 90
Height: 5
Types: ['water']
Abilities: ['rain-dish', 'torrent']



### Instance Methods

3. Write an instance method within the class `Pokemon` to find the BMI of a Pokemon. BMI is defined by $\frac{weight}{height^{2}}$ with weight in **kilograms** and height in **meters**. The height and weight data of Pokemon from the API is in **decimeters** and **hectograms** respectively.


    1 decimeter = 0.1 meters
    1 hectogram = 0.1 kilograms

In [99]:
# run this cell to test and check your code
# you will probably have to rerun the code to instantiate your objects

print(bulbasaur.bmi()) # 14.08
print(charmander.bmi()) # 23.61
print(squirtle.bmi()) # 36

14.08
23.61
36.0


## Part 3: SQL and Relational Databases

For this section, we've put the Pokemon data into SQL tables. You won't need to use your list of dictionaries or the JSON file for this section. The schema of `pokemon.db` is as follows:

<img src="data/pokemondb.png" alt="db schema" style="width:500px;"/>

Assign your SQL queries as strings to the variables `q1`, `q2`, etc. and run the cells at the end of this section to print your results as Pandas DataFrames.

- q1: query all columns from `Pokemon` the Pokemon that have base_experience above 200  

  
- q2: query the id, name, type1 and type2 of Pokemon that have **water** types as either their first or second type


- q3: query the average weight of Pokemon by their first type in descending order


- q4: query the Pokemon name, Pokemon type2, and what **type2** has "2xdamage" to


- q5: query the top 5 most common type1s, the minimum height, maximum height, minimum weight and maximum weight of pokemon with those type1s, and what associated type they do "0.5xdamage" to


**Important note on syntax**: use `double quotes ""` when quoting strings **within** your query and wrap the entire query in `single quotes ''` For the column titles that begin with numbers, you need to wrap the column names in double quotes.

In [134]:
cnx = sqlite3.connect('data/pokemon.db')

In [140]:
q1 = 'SELECT * FROM pokemon WHERE base_experience > 200 ORDER BY base_experience'
pd.read_sql(q1, cnx)

,id,name,base_experience,weight,height,type1,type2
0,18,pidgeot,216,395,15,normal,flying
1,36,clefable,217,400,13,fairy,None
2,26,raichu,218,300,8,electric,None
3,45,vileplume,221,186,12,grass,poison
4,71,victreebel,221,155,17,grass,poison
5,76,golem,223,3000,14,rock,ground
6,65,alakazam,225,480,15,psychic,None
7,94,gengar,225,405,15,ghost,poison
8,31,nidoqueen,227,600,13,poison,ground
9,34,nidoking,227,620,14,poison,ground


In [141]:
q2 = """SELECT id, name, type1, type2 FROM pokemon WHERE type1 = 'water' OR type2 ='water'"""
pd.read_sql(q2, cnx)

,id,name,type1,type2
0,7,squirtle,water,None
1,8,wartortle,water,None
2,9,blastoise,water,None
3,54,psyduck,water,None
4,55,golduck,water,None
5,60,poliwag,water,None
6,61,poliwhirl,water,None
7,62,poliwrath,water,fighting
8,72,tentacool,water,poison
9,73,tentacruel,water,poison


In [142]:
q3 = 'SELECT type1, ROUND(AVG(weight),2) avg_weight FROM pokemon GROUP BY type1 ORDER BY avg_weight DESC'
pd.read_sql(q3, cnx)

,type1,avg_weight
0,rock,876.11
1,dragon,766.00
2,water,579.68
3,fighting,542.86
4,psychic,515.63
5,normal,500.86
6,fire,480.25
7,ice,480.00
8,ground,452.63
9,electric,317.89


In [143]:
q4 = 'SELECT a.name, a.type2, b."2xdamage" FROM pokemon a, types b WHERE a.type2 = b.name'
pd.read_sql(q4, cnx)

,name,type2,2xdamage
0,bulbasaur,poison,grass
1,ivysaur,poison,grass
2,venusaur,poison,grass
3,charizard,flying,fighting
4,butterfree,flying,fighting
5,weedle,poison,grass
6,kakuna,poison,grass
7,beedrill,poison,grass
8,pidgey,flying,fighting
9,pidgeotto,flying,fighting


In [144]:
q5 = """SELECT a.type1, COUNT(a.type1) most_type1, MIN(a.height) min_height, MAX(a.height) max_height, 
        MIN(a.weight) min_weight, MAX(a.weight) max_weight, b."0.5xdamage"
        FROM pokemon a, types b 
        WHERE a.type1 = b.name
        GROUP BY type1 ORDER BY most_type1 DESC LIMIT 5"""
pd.read_sql(q5, cnx)

,type1,most_type1,min_height,max_height,min_weight,max_weight,0.5xdamage
0,water,28,3,65,40,2350,steel
1,normal,22,3,22,18,4600,None
2,poison,14,4,35,10,650,fighting
3,bug,12,3,15,29,560,fighting
4,fire,12,6,20,85,1550,bug


## Section 4: Web Scraping

### Accessing Data Using BeautifulSoup

Use BeautifulSoup to get quotes, authors, and tags from [Quotes to Read](http://quotes.toscrape.com/).

Before answering these questions, go to the site and inspect the page. Make sure to look at what links there are and how the site is structured.

1. Get the first author and the path for the author's page as a tuple from the [homepage](http://quotes.toscrape.com/).

In [214]:
# Make a get request to retrieve the page
html_page = requests.get('http://quotes.toscrape.com/') 
# Pass the page contents to beautiful soup for parsing
soup = BeautifulSoup(html_page.content, 'html.parser')

# Your code here
#after inspecting the page, I am accessing the info with name and href
search = soup.find_all('span')[1]
print(search)

<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>


In [317]:
# search = list(soup.children)[2]
search_list = list(search.children)
search_list

['by ',
 <small class="author" itemprop="author">Albert Einstein</small>,
 '\n',
 <a href="/author/Albert-Einstein">(about)</a>,
 '\n']

In [224]:
#codes to return the name
for_name = search_list[1]
name = list(for_name)
name

['Albert Einstein']

In [ ]:
#codes to return the href
for_page = search_list[3]
for_page['href']

In [307]:
#create a function to return one author as a dictionary
def find_name(url):
    # Make a get request to retrieve the page
    html_page = requests.get(url) 
    # Pass the page contents to beautiful soup for parsing
    soup = BeautifulSoup(html_page.content, 'html.parser')

    # Your code here
    #after inspecting the page, I am accessing the info with name and href
    search = soup.find_all('span')[1]
    search_list = list(search.children)
    for_name = search_list[1]
    name = list(for_name)[0]
    for_page = search_list[3]
    for_page_used = for_page['href']
    
    dict_auth = {name:for_page_used}
    return dict_auth

In [308]:
find_name('http://quotes.toscrape.com/')

{'Albert Einstein': '/author/Albert-Einstein'}

2. Write a function to get **all** the authors and href links for the authors from the [homepage](http://quotes.toscrape.com/)


In [320]:
def authors(url):
    '''
    input: url
    
    return: a dictionary of of authors and their urls
            {'author_1':'url_of_author_1', 'author_2':'url_of_author_2' ...}
    '''
    # Make a get request to retrieve the page
    html_page = requests.get(url) 
    # Pass the page contents to beautiful soup for parsing
    soup = BeautifulSoup(html_page.content, 'html.parser')
    
    author = {}
    for i in list(range(0,20)):
        if i % 2 != 0:
            search = soup.find_all('span')[i]
            search_list = list(search.children)
            for_name = search_list[1]
            name = list(for_name)[0]
            for_page = search_list[3]
            for_page_used = for_page['href']

            dict_auth = {name:for_page_used}
            author.update(dict_auth)
    return author

In [321]:
# run this cell to test the function
print(authors('http://quotes.toscrape.com/'))
print('\n')
print(authors('http://quotes.toscrape.com/page/3'))

{'Albert Einstein': '/author/Albert-Einstein', 'J.K. Rowling': '/author/J-K-Rowling', 'Jane Austen': '/author/Jane-Austen', 'Marilyn Monroe': '/author/Marilyn-Monroe', 'André Gide': '/author/Andre-Gide', 'Thomas A. Edison': '/author/Thomas-A-Edison', 'Eleanor Roosevelt': '/author/Eleanor-Roosevelt', 'Steve Martin': '/author/Steve-Martin'}


{'Pablo Neruda': '/author/Pablo-Neruda', 'Ralph Waldo Emerson': '/author/Ralph-Waldo-Emerson', 'Mother Teresa': '/author/Mother-Teresa', 'Garrison Keillor': '/author/Garrison-Keillor', 'Jim Henson': '/author/Jim-Henson', 'Dr. Seuss': '/author/Dr-Seuss', 'Albert Einstein': '/author/Albert-Einstein', 'J.K. Rowling': '/author/J-K-Rowling', 'Bob Marley': '/author/Bob-Marley'}


### Pagination

3. Get the first author on each of the first 5 pages of quotes. You can get to the next page with the next button at the bottom of the homepage.


In [ ]:
# Your code here


4. Write a function to get all of the quotes from a page.

In [ ]:
def get_some_quotes(url):
    '''
    input: url, number of pages to scrap (just scrape the home page if no argument is passed in)
    
    return: a list of dictionaries of quotes with their attributes
            [{'quote':'quote_1_text', 'author':'url_of_author_1'}, 
            {'quote':'quote_2_text', 'author':'url_of_author_2', 'quote_tags':[list_of_quote_2_tags]}, ...]
    '''
    pass

In [ ]:
# set the function to a variable to use later
quotes_for_mongo = get_some_quotes('http://quotes.toscrape.com/' )
quotes_for_mongo

## Part 5: MongoDB

To do this section, open a connection to a mongo database in the terminal, using `mongod`. You will then **create**, **update**, and **read** from a mongo database.

Create and connect to a mongo database.

In [ ]:
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
mydb = myclient['quote_database']

In [ ]:
mycollection = mydb['quote_collection']

1. Add the quotes you obtained from the `get_some_quotes` function for the [homepage](http://quotes.toscrape.com/) to the mongo database. (You can also use the JSON file `quotes.json` to insert data into the database) To verify that you've successfully inserted the data, query it to obtain the resulting _ids back from the `results` variable. 

In [ ]:
# if you were unable to get the data from webscraping in the right format,
# uncomment the code below to access a JSON file with the list of dictionaries

# with open(r"data/quotes.json", "r") as r:
#     data = json.load(r)

In [ ]:
# use the results variable to confirm the data was inserted
results = None

2. Query the database for all the quotes written by `'Albert Einstein'`.

In [ ]:
q1 = None

3. Update Steve Martin's quote with the tags for the quote stored in the variable `steve_martin_tags`.

In [ ]:
steve_martin_tags = {'quote_tags': ['change', 'deep-thoughts', 'thinking', 'world']}
update_steve = None
first_quote_tags = None


4. Query the database to confirm that  `'Steve Martin'` is updated with `steve_martin_tags`.

In [ ]:
q2 = None